# 워크넷 API 이용 기업 정보 데이터 베이스화 

간단 요약 설명 : 워크넷 open api 를 이용해서 채용정보를 얻어 오려고 한다. 
채용정보직종은 데이터 분석과 관련된 직종이며 상세하게 얻어오려고 한다. 
파이썬으로는 얻어온 정보를 정리하고 + 카카오api를 이용해 좌표까지 얻어와서 
csv파일로 만드는 것까지가 목표( 시각화를 위한 tableau 활용 예정) $$ 

1. 모듈 설치

In [8]:
import requests     # url 이용해서 정보 얻기 
import pandas as pd # pandas 데이터프레임화해서 자료정리 
import numpy as np
from bs4 import BeautifulSoup # 뷰티풀숲으로 자료 원하는 것만 추출 
import json


2. 검색할 직업 목록 확인 (데이터 관련직만 확인해보자)

In [9]:
code = pd.read_csv('/Users/snoo/enterprise/직종코드1.csv',encoding='utf-8',
                   index_col='Unnamed: 0') 
code.columns = ['직종코드','대분류','중분류','소분류']
code

,직종코드,대분류,중분류,소분류
0,NaN,NaN,NaN,NaN
1,카테고리 ID,1 depth,2 depth,3_depth
2,01,경영·사무·금융·보험,,
3,011,,행정·경영·금융·보험 관리직,
4,011100,,,의회의원·고위공무원 및 공공단체임원
...,...,...,...,...
1294,135,,농림어업 단순 종사자,
1295,905000,,,농림어업 단순 종사원
1296,905001,,,농업 단순 종사원
1297,905002,,,"임업 단순 종사원(산림보호감시, 산불감시원 등)"


2. 1) 부분 문자열 검사를 통해 직종 코드 찾기 

In [10]:
code[code['소분류'].str.contains('데이터',na=False)] 

,직종코드,대분류,중분류,소분류
249,134100,,,데이터 전문가
250,134101,,,데이터 설계 및 프로그래머
251,134102,,,데이터 분석가(빅데이터 분석가)
252,134103,,,데이터베이스 운영·관리자
258,134900,,,디지털 포렌식 전문가 및 기타 데이터·네트워크 전문가


In [11]:
# db 관리자나 디지털포렌식 전문가는 원하는 직종이 아니기 때문에 제외하도록 하자
need_code = code['직종코드'][code['소분류'].str.contains('데이터',na=False)]
need_code = need_code[:3]

3. 워크넷 open api 사용법 
    - 워크넷 오픈api 사이트 회원가입 필수 (인증키 신청) 
    - 인증키 get 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=(인증키)&callTp=L&returnType=XML&startPage=1&display=100&occupation=134100'

인증키 = 인증키 
callTp = 얻을 정보 기본 L, 상세 = D 상세는 밑에 다룰 예정 
returnType = XML 
startPage = 시작페이지 
display = 최대 100
occupation = 직종코드
    



In [12]:
need_code.index = range(0,3)
need_code[0]

'134100'

In [13]:
# 먼저 하나만 먼저 해보도록한다
key = pd.read_csv('/Users/snoo/key/worknet_key.txt').columns[0]
# key = pd.read_csv('C:/SEONWOO/enterprise/worknet_key.txt').columns[0]
url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?\
authKey='+key+'&callTp=L&returnType=XML&startPage=1&\
display=100&occupation='+need_code[0]
print(url)

http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=WNL596OLQCP7023TCKJPB2VR1HJ&callTp=L&returnType=XML&startPage=1&display=100&occupation=134100


4. request 모듈을 사용해서 정보 얻기

In [14]:
res = requests.get(url)
text = res.text 


In [15]:
# 뷰티풀숲의 find_all 같은 기능으로 원하는 정보만 얻을 생각이다.
soup = BeautifulSoup(text,'lxml-xml')

# display = 94 총 94개의 구직 정보가 있음 

5. 뷰티풀숲을 이용해 구인인증번호('wantedAuthNo')를 얻을 생각이다. 
 why? / 상세를 보기 위해서는 구인인증번호가 필요하다. 

In [16]:
# 전체의 구인인증번호를 반복문을 통해서 리스트로 만들 생각이다. 
soup.find_all('wantedAuthNo')[0].text 

'K120252207110024'

In [18]:
guin_num = []
for i in range(int(soup.find('display').text)):
    guin_num.append(soup.find_all('wantedAuthNo')[i].text)
len(guin_num)

94

## 직종코드 3가지를 url에 넣고 구인인증번호를 하나의 리스트로 저장하자 

In [19]:
key = pd.read_csv('/Users/snoo/key/worknet_key.txt').columns[0]
# key = pd.read_csv('C:/SEONWOO/enterprise/worknet_key.txt').columns[0]
guin_num_all = []

for i in range(len(need_code)):
    url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?\
authKey='+key+'&callTp=L&returnType=XML&startPage=1&\
display=100&occupation='+need_code[i]
    res = requests.get(url)
    text = res.text
    soup = BeautifulSoup(text,'lxml-xml')
    for j in range(int(soup.find('display').text)):
        guin_num_all.append(soup.find_all('wantedAuthNo')[j].text)

len(guin_num_all)



157

#### 상세정보 얻는 url  방식은 위와 같다. guin_num_all 의 0번을 먼저 확인하자.
http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=[인증키]&callTp=D&returnType=XML
&wantedAuthNo=[구인인증번호]&infoSvc=VALIDATION

In [20]:
key = pd.read_csv('/Users/snoo/key/worknet_key.txt').columns[0]
# key = pd.read_csv('C:/SEONWOO/enterprise/worknet_key.txt').columns[0]
url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey='+key+\
'&callTp=D&returnType=XML&wantedAuthNo='+guin_num_all[0]+'&infoSvc=VALIDATION'
print(url)

http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey=WNL596OLQCP7023TCKJPB2VR1HJ&callTp=D&returnType=XML&wantedAuthNo=K120252207110024&infoSvc=VALIDATION


In [21]:
res = requests.get(url)
text = res.text 

In [22]:
soup = BeautifulSoup(text,'lxml-xml')

## 구인정보 상세   필요한것만 넘버링
1. <corpNm>	String	회사명	</corpNm>
<reperNm>	String	대표자명	</reperNm>
2. <totPsncnt>	Number	근로자수	</totPsncnt>
3. <capitalAmt>	Number	자본금	</capitalAmt>
4. <yrSalesAmt>	Number	연매출액	</yrSalesAmt>
5. <indTpCdNm>	String	업종	</indTpCdNm>
6. <busiCont>	String	주요사업내용	</busiCont>
7. <corpAddr>	String	회사주소	</corpAddr>
<homePg>	String	회사홈페이지	</homePg>
8. <busiSize>	String	회사규모	</busiSize>
9. <jobsNm>	String	모집집종	</jobsNm>
10. <wantedTitle>	String	구인제목	</wantedTitle>
<relJobsNm>	String	관련직종	</relJobsNm>
11. <jobCont>	String	직무내용	</jobCont>
12. <receiptCloseDt>	String	접수마감일	</receiptCloseDt>
13. <empTpNm>	String	고용형태	</empTpNm>
14. <collectPsncnt>	String	모집인원	</collectPsncnt>
15. <salTpNm>	String	임금조건	</salTpNm>
16. <enterTpNm>	String	경력조건	</enterTpNm>
17. <eduNm>	String	학력	</eduNm>
18. <forLang>	String	외국어	</forLang>
19. <major>	String	전공	</major>
20. <certificate>	String	자격면허	</certificate>
21. <mltsvcExcHope>	String	병역특례채용희망	</mltsvcExcHope>
22. <compAbl>	String	컴퓨터활용능력	</compAbl>
23. <pfCond>	String	우대조건	</pfCond>
24. <etcPfCond>	String	기타우대조건	</etcPfCond>
25. <selMthd>	String	전형방법	</selMthd>
26. <rcptMthd>	String	접수방법	</rcptMthd>
27. <submitDoc>	String	제출서류준비물	</submitDoc>
28. <etcHopeCont>	String	기타안내	</etcHopeCont>
29. <workRegion>	String	근무예정지	</workRegion>
30. <nearLine>	String	인근전철역	</nearLine>
31. <workdayWorkhrCont>	String	근무시간/형태	</workdayWorkhrCont>
32. <fourIns>	String	연금4대 보험	</fourIns>
33. <retirepay>	String	퇴직금	</retirepay>
34. <etcWelfare>	String	기타복리후생	</etcWelfare>
35. <disableCvntl>	String	장애인편의시설	</disableCvntl>
36. <dtlRecrContUrl>	String	상세모집내용 URL	</dtlRecrContUrl>
 <minEdubgIcd>	String	최소학력코드 </minEdubgIcd>
<regionCd>	String	근무지역코드	</regionCd>
<empTpCd>	String	고용형태코드  </empTpCd>
<enterTpCd>	String	근무형태코드  </enterTpCd>
<salTpCd>	String	임금형태코드</salTpCd>
<staAreaRegionCd>	String	근무지지하철지역코드	</staAreaRegionCd>
<walkDistCd>	String	근무지지하철출구거리 </walkDistCd>
37. <contactTelno>	String	전화번호	</contactTelno>





In [24]:
+821067712201info = ['corpNm','totPsncnt','capitalAmt','yrSalesAmt','indTpCdNm','busiCont',
     +821067712201  'corpAddr','busiSize','jobsNm','wantedTitle','jobCont','receiptCloseDt',
       'empTpNm','collectPsncnt','salTpNm','enterTpNm','eduNm','forLang',
       'major','certificate','mltsvcExcHope','compAbl','pfCond','etcPfCond',
       'selMthd','rcptMthd','submitDoc','etcHopeCont','workRegion','nearLine',
       'workdayWorkhrCont','fourIns','retirepay','etcWelfare','disableCvntl',
       'dtlRecrContUrl','contactTelno']
info

['corpNm',
 'totPsncnt',
 'capitalAmt',
 'yrSalesAmt',
 'indTpCdNm',
 'busiCont',
 'corpAddr',
 'busiSize',
 'jobsNm',
 'wantedTitle',
 'jobCont',
 'receiptCloseDt',
 'empTpNm',
 'collectPsncnt',
 'salTpNm',
 'enterTpNm',
 'eduNm',
 'forLang',
 'major',
 'certificate',
 'mltsvcExcHope',
 'compAbl',
 'pfCond',
 'etcPfCond',
 'selMthd',
 'rcptMthd',
 'submitDoc',
 'etcHopeCont',
 'workRegion',
 'nearLine',
 'workdayWorkhrCont',
 'fourIns',
 'retirepay',
 'etcWelfare',
 'disableCvntl',
 'dtlRecrContUrl',
 'contactTelno']

In [25]:
soup.find('corpNm').text

'알투웨어(주)'

In [26]:
soup.find(info[5]).text

'소프트웨어개발 및 기술인력파견'

In [27]:
worknet_df = pd.DataFrame(columns=['회사명','근로자수','자본금','연매출액',
                                  '업종','주요사업내용','회사주소','회사규모',
                                  '모집직종','구인제목','직무내용','접수마감일','고용형태',
                                  '모집인원','임금조건','경력조건','학력','외국어','전공',
                                  '자격면허','병역특례채용희망','컴활능력','우대조건','기타우대조건',
                                  '전형방법','접수방법','제출서류준비물','기타안내','근무예정지',
                                  '인근전철역','근무시간/형태','연금4대보험','퇴직금','기타복리후생','장애인편의시설',
                                  '상세모집내용URL','전화번호'])

worknet_df

,회사명,근로자수,자본금,연매출액,업종,주요사업내용,회사주소,회사규모,모집직종,구인제목,...,기타안내,근무예정지,인근전철역,근무시간/형태,연금4대보험,퇴직금,기타복리후생,장애인편의시설,상세모집내용URL,전화번호


In [28]:
# 상세 정보에 없는 값이 있어서 AttributeError 발생 해서 예외처리 작업 실시 , 오류 발생 시 nan 값을 집어넣었다.
key = pd.read_csv('/Users/snoo/key/worknet_key.txt').columns[0]

for i in guin_num_all:
    url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do?authKey='+key+\
    '&callTp=D&returnType=XML&wantedAuthNo='+i+'&infoSvc=VALIDATION'
    res = requests.get(url)
    text = res.text
    soup = BeautifulSoup(text,'lxml-xml')
    info_list = []
    for j in info:
        try:        
            info_list.append(soup.find(j).text)
        except AttributeError:
            info_list.append(np.nan)
    worknet_df = worknet_df.append(pd.Series(info_list,index=worknet_df.columns),ignore_index=True)
worknet_df

,회사명,근로자수,자본금,연매출액,업종,주요사업내용,회사주소,회사규모,모집직종,구인제목,...,기타안내,근무예정지,인근전철역,근무시간/형태,연금4대보험,퇴직금,기타복리후생,장애인편의시설,상세모집내용URL,전화번호
0,알투웨어(주),30 명,210 백만원,80000 백만원,시스템 소프트웨어 개발 및 공급업,소프트웨어개발 및 기술인력파견,"04788 서울특별시 성동구 광나루로 130, 서울숲IT캐슬 303호 (성수동1가)",중소기업,IT 기술지원 전문가(134303),[Software Engineer] 알투웨어 기술지원팀 채용 (주4.5일),...,,"(04788) 서울특별시 성동구 광나루로 130, 서울숲IT캐슬 303호 (성수동1가)",2호선 뚝섬 1번출구 500M,"평일 : 주 5일(월~목) 09:00~18:00 (금) 09:00~14:00, 주 ...",국민연금 고용보험 산재보험 의료보험,퇴직연금,"기타(4.5일 근무제, 건강검진, 자기개발비, 경조사비, 4대보험, 퇴직연금, 명절...",,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-422-8005
1,제주한라대학산학협력단,25 명,0 백만원,0 백만원,교육 서비스업,교육서비스업,63092 제주특별자치도 제주시 한라대학로 38 (노형동),기타,데이터 분석가(빅데이터 분석가)(134102),제주한라대학교 산학협력단(지역특화산업육성+[R&D]-지역주력산업육성)미래문화기술연 ...,...,,(63092) 제주특별자치도 제주시 한라대학로 38 (노형동),,"평일 : (오전) 9시 00분~(오후) 5시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,해당없음,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,064-741-7476
2,(주)베이소프트,3 명,0 백만원,0 백만원,컴퓨터 프로그래밍 서비스업,"데이터분석, 소프트웨어개발, SI","06693 서울특별시 서초구 방배천로2길 21, 4층 410호 (방배동)",,웹 개발자(웹 엔지니어·웹 프로그래머)(133301),IT개발·데이터 정규직/계약직 채용합니다,...,,"(06693) 서울특별시 서초구 방배천로2길 21, 4층 410호 (방배동)",4호선 사당 14번출구 200M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,070-4116-2876
3,스페이스플래닝,2 명,1 백만원,0 백만원,포털 및 기타 인터넷 정보매개 서비스업,인테리어 빅데이터 플랫폼,"61472 광주광역시 동구 금남로 193-12, AI창업캠프2호점 303호 (금남로4가)",,데이터 분석가(빅데이터 분석가)(134102),데이터 분석 담당자 구인합니다,...,스톡옵션 등 초기 인원으로서의 메리트를 드리고자 합니다 \r\n데이터 관련해서 전문...,"(61472) 광주광역시 동구 금남로 193-12, AI창업캠프2호점 303호 (...",1호선 금남로4가 3번출구 10M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,062-225-2224
4,(주)씨앤씨솔루션,8 명,NaN,NaN,그 외 기타 특수목적용 기계 제조업,수질환경,"46751 부산광역시 강서구 녹산산업중로 62-6, C&C SOLUTION",중소기업,데이터 설계 및 프로그래머(134101),기계 설계 경력직 채용,...,,"(46751) 부산광역시 강서구 녹산산업중로 62-6, C&C SOLUTION",,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,차량유지비,,http://www.work.go.kr/outOffer/empInfo/empDeta...,051-941-2814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,(주)한솥,35 명,105 백만원,99590 백만원,기타 가공식품 도매업,도시락 체인본부,"135937 서울특별시 강남구 강남대로 318, 타워837빌딩 8,9층 (역삼동)",중견기업,데이터 전문가(134100),(주)한솥 디지털 기획 운영팀 IT담당 경력사원 채용,...,,"(135937) 서울특별시 강남구 강남대로 318, 타워837빌딩 8,9층 (역삼동)",신분당선 강남 4번출구 200M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,"기타(건강검진, 경조휴가, 경조금, 자녀학자금, 휴양소(법인콘도), 독감예방접종, ...",,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2033-9632
153,나이스지니데이타,56 명,0 백만원,0 백만원,데이터베이스 및 온라인정보 제공업,데이터 분석 및 판매,"07242 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신관) ...",중견기업,데이터 분석가(빅데이터 분석가)(134102),[NICE 지니데이타] 빅데이터분석 컨설팅 신입직원 채용,...,,"(07242) 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신...",9호선 국회의사당 3번출구 200M,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,교육비 지원,,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2639-2398
154,나이스지니데이타,56 명,0 백만원,0 백만원,데이터베이스 및 온라인정보 제공업,데이터 분석 및 판매,"07242 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신관) ...",중견기업,데이터 분석가(빅데이터 분석가)(134102),[NICE 지니데이타] 빅데이터분석 컨설팅 경력직원 채용,...,,"(07242) 서울특별시 영등포구 은행로 30, 7층(여의도동, 중소기업중앙회 신...",,"평일 : (오전) 9시 00분~(오후) 6시 00분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,02-2639-2398
155,(주)지오비전,40 명,202 백만원,799 백만원,응용 소프트웨어 개발 및 공급업,지능형 영상분석 솔루션,"24341 강원도 춘천시 강원대학길 1, 보듬관 404, 405호",중소기업,시스템 소프트웨어 개발자(133100),데이터 분석 및 빅데이터 기반 딥러닝 경력/석사 채용(서울/춘천),...,,"(24341) 강원도 춘천시 강원대학길 1, 보듬관 404, 405호",2호선 구로디지털단지 2번출구 800M,"평일 : (오전) 9시 30분 ~ (오후) 6시 30분, 주 5일 근무, 평균근무시...",국민연금 고용보험 산재보험 의료보험,퇴직금,"기타(- 주 5일 근무 ( 연차,반차 ) -- 주5일근무시간 (탄력출퇴근제)\r\...",,http://www.work.go.kr/outOffer/empInfo/empDeta...,


In [29]:
worknet_df.shape

(157, 37)

In [30]:
worknet_df.to_csv('/Users/snoo/enterprise/worknet_df.csv',index=False)

In [61]:
worknet_df =pd.read_csv('/Users/snoo/enterprise/worknet_df.csv')

In [66]:
worknet_df1 = worknet_df.copy()

In [67]:
worknet_df1.isnull().sum()

회사명            0
근로자수           0
자본금            7
연매출액           9
업종             0
주요사업내용         0
회사주소           0
회사규모          27
모집직종           0
구인제목           0
직무내용           0
접수마감일          0
고용형태           0
모집인원           0
임금조건           0
경력조건           0
학력             0
외국어          140
전공            86
자격면허          92
병역특례채용희망       0
컴활능력          82
우대조건         120
기타우대조건       127
전형방법           0
접수방법           0
제출서류준비물        1
기타안내         143
근무예정지          0
인근전철역          0
근무시간/형태        0
연금4대보험         0
퇴직금            0
기타복리후생       111
장애인편의시설      156
상세모집내용URL      0
전화번호          22
dtype: int64

In [68]:
# 태블로우에서 nan 값이나 ''값은 인식 못해서 스페이스 하나 넣어줬다. = 태블로우에선 Null 로 나온다. 
# 이제 주소만 따서 좌표값 추가해주고 바꿔줄거 자본금, 매출 등등 숫자형태로 바꿔줘야겠다. 
worknet_df1 = worknet_df1.fillna(' ')
worknet_df1.to_csv('/Users/snoo/enterprise/worknet_df1.csv',index=False)

In [69]:
worknet_df1.loc[2]

회사명                                                   (주)베이소프트
근로자수                                                       3 명
자본금                                                      0 백만원
연매출액                                                     0 백만원
업종                                              컴퓨터 프로그래밍 서비스업
주요사업내용                                      데이터분석, 소프트웨어개발, SI
회사주소                  06693 서울특별시 서초구 방배천로2길 21, 4층 410호 (방배동)
회사규모                                                          
모집직종                             웹 개발자(웹 엔지니어·웹 프로그래머)(133301)
구인제목                                    IT개발·데이터 정규직/계약직 채용합니다
직무내용         &lt;&lt; 웹 개발자 &gt;&gt;\r\n담당업무\r\nㆍ웹 개발\r\n- ...
접수마감일                                                    채용시까지
고용형태                     기간의 정함이 없는 근로계약/ 파견근로 비희망/ 대체인력채용 비희망
모집인원                                                         3
임금조건                        연봉23,000,000원 이상 ~ 70,000,000원 이하,
경력조건                                                   

#### kakao API 이용 주소를 이용 좌표로 변경 (tableau 맵을 이용해 회사 위치 파악)

In [36]:
#kakao_key = pd.read_csv('C:/SEONWOO/enterprise/key.txt').columns[0]
kakao_key = pd.read_csv('/Users/snoo/key/key.txt').columns[0]

좌표 얻는 함수 

In [37]:
def getLatLng(addr):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
  headers = {"Authorization": "KakaoAK "+kakao_key}
  result = json.loads(str(requests.get(url,headers=headers).text))
  match_first = result['documents'][0]['address']
  return float(match_first['y']),float(match_first['x'])

In [39]:
#worknet_df = pd.read_csv('C:/SEONWOO/enterprise/worknet_df1.csv')
# 맥북버전 
worknet_df = pd.read_csv('/Users/snoo/enterprise/worknet_df1.csv')
worknet_df['회사주소'] # 형태 우편번호 + 주소 / 우리는 ㅇㅇ로 ㅁㅁㅁ 까지만 필요하니 분리해서 작업해야겠다. 
import re
juso = worknet_df['회사주소']
' '.join(juso[0].split(' ')[1:]).split(',')[0] # 주소 추출 

'서울특별시 성동구 광나루로 130'

In [40]:
최juso = list(map(lambda x : ' '.join(x.split(' ')[1:]).split(',')[0],juso))

coord= []
for i in range(len(worknet_df)):
    coord.append(getLatLng(juso[i]))

In [41]:
coord

[(37.5509787653401, 127.044720723884),
 (33.4767897566467, 126.474151903132),
 (37.4772565293096, 126.983878760744),
 (35.1514000679446, 126.914752917327),
 (35.0971635156665, 128.827780932198),
 (37.4488513932536, 127.167696591792),
 (35.2161116157406, 128.584495446582),
 (35.534228487564, 129.337146711287),
 (35.8682069453509, 128.546093759952),
 (37.4168272565636, 126.86085676081),
 (37.3225456341018, 127.098547188991),
 (36.4272972175849, 127.392562507138),
 (35.1567141271262, 128.098001405271),
 (37.5791229754801, 126.869769504006),
 (37.568851450649, 126.827338682565),
 (36.3584119241385, 127.348660929428),
 (37.5887034223667, 127.031698331241),
 (37.5637512316327, 126.964632683146),
 (35.701755242576, 128.466150981391),
 (37.3996441086743, 126.970274096871),
 (37.5451797442107, 127.056896775555),
 (37.3537312484956, 126.957422229723),
 (37.4795130993704, 126.887511773421),
 (35.1180568683281, 129.04218656028),
 (37.1779083926247, 126.883796738318),
 (37.5341520566604, 127.006517

In [42]:
coord = list(map(lambda x : str(x).split('(')[1].split(')')[0],coord))
coord_y = list(map(lambda x : x.split(',')[0],coord))
coord_x = list(map(lambda x : x.split(',')[1],coord))

In [70]:
worknet_df['위도'] = coord_y
worknet_df['경도'] = coord_x


## 임시 저장

In [71]:
# worknet_df.to_csv('C:/SEONWOO/enterprise/enterprise/worknet_df1.csv',encoding='utf-8')
worknet_df.to_csv('/Users/snoo/enterprise/worknet_df1.csv',index=False)

In [72]:
import pandas as pd
worknet_df = pd.read_csv('/Users/snoo/enterprise/worknet_df1.csv')

In [77]:
worknet_df.isnull().sum()
worknet_df = worknet_df.fillna(' ')

In [79]:
worknet_df.to_csv('/Users/snoo/enterprise/worknet_df1.csv',index=False)

### 변경이 필요한 컬럼이 있는가?? 
- 근로자수, 자본금, 연매출액,수치로 두는것이 좋을것 같다. 
- 임금조건 = 임금형태/ 임금
- 접수마감일 날짜형태로 / 채용시까지 = 20221231 로 변경 


In [81]:
worknet_df1 = pd.read_csv('/Users/snoo/enterprise/worknet_df1.csv')

In [82]:
worknet_df1['근로자수'] = list(map(lambda x : int(x.replace('명','')),worknet_df['근로자수']))
worknet_df1['근로자수'].head()

0    30
1    25
2     3
3     2
4     8
Name: 근로자수, dtype: int64

In [89]:
최선우worknet_df1['자본금'] = list(map(lambda x : x.replace(' 백만원','000000'),worknet_df1['자본금']))
worknet_df1['자본금'] = list(map(lambda x : x.replace(' ','0'),worknet_df1['자본금']))
worknet_df1['자본금'] = worknet_df1['자본금'].astype('int')
worknet_df1['자본금'].head()

0    210000000
1            0
2            0
3      1000000
4            0
Name: 자본금, dtype: int64

In [90]:
최선우worknet_df1['연매출액'] = list(map(lambda x : int(x.replace(' 백만원','000000').replace(' ','0')),worknet_df1['연매출액']))
worknet_df1['연매출액'].head()

0    80000000000
1              0
2              0
3              0
4              0
Name: 연매출액, dtype: int64

In [91]:
최선우worknet_df1['임금형태'] =list(map(lambda x : x.split('이상')[0][:2],worknet_df1['임금조건']))
worknet_df1['임금'] = list(map(lambda x : int(x.split('이상')[0][2:].replace(',','')[:-2]),worknet_df1['임금조건']))
worknet_df1.drop('임금조건',axis=1,inplace=True)
worknet_df1['임금형태'].head()

0    연봉
1    연봉
2    연봉
3    연봉
4    연봉
Name: 임금형태, dtype: object

In [92]:
최선우worknet_df1['임금'].head()

0    33000000
1    20060400
2    23000000
3    28000000
4    40000000
Name: 임금, dtype: int64

In [93]:
최선우worknet_df1['접수마감일'] = list(map(lambda x : x.replace('채용시까지','20221231'),worknet_df1['접수마감일']))
import datetime
worknet_df1['접수마감일'] = list(map(lambda x : datetime.datetime.strptime(x,'%Y%M%d').strftime('%Y-%M-%d'),worknet_df1['접수마감일']))
worknet_df1['접수마감일'].head()

0    2022-12-31
1    2022-07-25
2    2022-12-31
3    2022-07-25
4    2022-07-23
Name: 접수마감일, dtype: object

중복이 한건 존재 했다 삭제 하고 저장하겠다.

In [2]:
import pandas as pd 
aa = pd.read_csv('/Users/snoo/enterprise/worknet_df2.csv')
aa[aa['회사명']=='영천제일직업전문학교']
aa.drop(138,inplace=True)

,회사명,근로자수,자본금,연매출액,업종,주요사업내용,회사주소,회사규모,모집직종,구인제목,...,연금4대보험,퇴직금,기타복리후생,장애인편의시설,상세모집내용URL,전화번호,위도,경도,임금형태,임금
40,영천제일직업전문학교,8,0,0,직원훈련기관,실업자 직업훈련 및 취업지원 업무,38888 경상북도 영천시 완산로 73 (완산동),중소기업,데이터 분석가(빅데이터 분석가)(134102),빅데이터 강사 구합니다.,...,국민연금 고용보험 산재보험 의료보험,해당없음,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,054-337-7676,35.966801,128.938304,시급,30000
138,영천제일직업전문학교,8,0,0,직원훈련기관,실업자 직업훈련 및 취업지원 업무,38888 경상북도 영천시 완산로 73 (완산동),중소기업,데이터 분석가(빅데이터 분석가)(134102),빅데이터 강사 구합니다.,...,국민연금 고용보험 산재보험 의료보험,해당없음,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,054-337-7676,35.966801,128.938304,시급,30000


In [7]:
aa.to_csv('/Users/snoo/enterprise/worknet_df2.csv',index=False)

데이터프레임에서 컬럼 적용은 여기 까지하면 될 것 같다. 나머지는 시각화에 달려있다.
csv 파일로 만든 후 tableau 를 이용해서 작업을 해 보자 